In [1]:
import numpy as np 
import pandas as pd
import cvxopt
from cvxopt import matrix
from cvxopt import solvers
import copy
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
arr =[]
for i in range(784):
  arr.append(i)
print(arr)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [4]:
df_inp = pd.read_csv("/content/drive/MyDrive/train.csv", header=None, usecols =arr)

In [ ]:
print(df_inp)

       0    1    2    3    4    5    ...    778    779    780   781  782  783
0      0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0   0.0  0.0  0.0
1      0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0   0.0  0.0  0.0
2      0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0   0.0  0.0  0.0
3      0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0   0.0  0.0  0.0
4      0.0  0.0  0.0  0.0  2.0  0.0  ...  208.0  234.0  207.0  99.0  0.0  0.0
...    ...  ...  ...  ...  ...  ...  ...    ...    ...    ...   ...  ...  ...
22495  0.0  0.0  0.0  0.0  0.0  0.0  ...  179.0  182.0    0.0   0.0  0.0  0.0
22496  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0   0.0  0.0  0.0
22497  0.0  0.0  0.0  0.0  0.0  0.0  ...    2.0    0.0    0.0   0.0  0.0  0.0
22498  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0   0.0  0.0  0.0
22499  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0    0.0    0.0   0.0  0.0  0.0

[22500 rows x 784 columns]


In [5]:
arr_t = [784]
df_labels = pd.read_csv("/content/drive/MyDrive/train.csv", header=None, usecols=arr_t)

In [ ]:
print(df_labels)

       784
0      5.0
1      1.0
2      7.0
3      7.0
4      6.0
...    ...
22495  1.0
22496  3.0
22497  3.0
22498  8.0
22499  7.0

[22500 rows x 1 columns]


In [6]:
np_X = df_inp.to_numpy()
np_Y = df_labels.to_numpy()
print(np_X)
print(np_Y)
np_X /= 255

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[5.]
 [1.]
 [7.]
 ...
 [3.]
 [8.]
 [7.]]


Setting up data for binary classification problem

In [7]:
c=0
for k in np_Y:
  if k==4 or k==5:
    c+=1
np_X_binary = np.zeros((c,784))
np_Y_binary = np.zeros(c)
o=0
#print(np.shape(np_Y)[0])
for k in range(np.shape(np_Y)[0]):
  if np_Y[k]==4 or np_Y[k]==5:
    np_Y_binary[o]=np_Y[k]
    np_X_binary[o]=np_X[k]
    o+=1
print(np_X_binary)
print(np_Y_binary)
for i in range(np.shape(np_Y_binary)[0]):
  if np_Y_binary[i]==4:
    np_Y_binary[i]=1
  else:
    np_Y_binary[i]=-1
print(np_Y_binary)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.06666667 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[5. 4. 4. ... 5. 5. 4.]
[-1.  1.  1. ... -1. -1.  1.]


Using Convetion for CVX OPT 

In [8]:
C=1.0
m = c
q_temp = np.ones((m,1), dtype=np.float64)
q_temp *= (-1)
a_temp = copy.deepcopy(np_Y_binary)
a_temp = a_temp.reshape(1,m)
a_temp = a_temp.astype(np.float64)
#print(a_temp)
#print(q_temp)
#print(np.shape(a_temp))
b_temp=0.0
h_temp = np.zeros((2*m,1), dtype=np.float64)
for j in range(m):
  h_temp[j]=C
idi = np.identity(m,dtype = np.float64)
idi_neg = copy.deepcopy(idi)
idi_neg *= (-1)
g_temp = np.concatenate((idi,idi_neg),axis=0)
p_temp = np.zeros((m,m), dtype=np.float64)
#print(np.dot(np.transpose(np_X_binary[500]),np_X_binary[500]))
for i in range(m):
  for j in range(i+1):
    use = (np_Y_binary[i].item()*np_Y_binary[j].item())*np.dot(np.transpose(np_X_binary[i]),np_X_binary[j])
    p_temp[i][j]=use
    p_temp[j][i]=use
#print(p_temp)


In [ ]:
print(np.shape(p_temp))
print(p_temp)

(4500, 4500)
[[ 69.27880046 -78.02674356 -82.81731642 ...  40.13450211  25.73548635
  -72.57034987]
 [-78.02674356 316.85187236 319.46674356 ... -82.80186082 -51.394802
  272.72544406]
 [-82.81731642 319.46674356 393.53053441 ... -90.84450596 -62.79194156
  310.69865436]
 ...
 [ 40.13450211 -82.80186082 -90.84450596 ...  66.38126874  39.53121107
  -81.74821992]
 [ 25.73548635 -51.394802   -62.79194156 ...  39.53121107  56.75483276
  -56.43737024]
 [-72.57034987 272.72544406 310.69865436 ... -81.74821992 -56.43737024
  283.84601307]]


Setting up all matrices for CVXOPT

In [9]:
P = matrix(p_temp)
q = matrix(q_temp)
G = matrix(g_temp)
h = matrix(h_temp)
A = matrix(a_temp)
b = matrix(b_temp)


In [10]:
sol = solvers.qp(P,q,G,h,A,b)

     pcost       dcost       gap    pres   dres
 0: -1.8144e+02 -7.2340e+03  4e+04  2e+00  2e-12
 1: -1.0203e+02 -3.4492e+03  6e+03  3e-01  1e-12
 2: -3.3534e+01 -9.1974e+02  2e+03  7e-02  8e-13
 3: -1.1925e+01 -4.1859e+02  7e+02  2e-02  4e-13
 4: -3.6847e+00 -1.7770e+02  3e+02  8e-03  2e-13
 5: -1.2594e+00 -5.9907e+01  9e+01  2e-03  6e-14
 6: -6.7211e-01 -2.5705e+01  4e+01  9e-04  3e-14
 7: -4.1327e-01 -1.1925e+01  2e+01  3e-04  2e-14
 8: -5.1263e-01 -5.5495e+00  6e+00  1e-04  2e-14
 9: -5.7700e-01 -3.1645e+00  3e+00  4e-05  2e-14
10: -7.0634e-01 -2.0084e+00  1e+00  3e-06  2e-14
11: -9.3593e-01 -1.4177e+00  5e-01  7e-07  2e-14
12: -1.0291e+00 -1.2146e+00  2e-01  2e-16  2e-14
13: -1.0863e+00 -1.1319e+00  5e-02  6e-16  2e-14
14: -1.1048e+00 -1.1100e+00  5e-03  4e-16  2e-14
15: -1.1073e+00 -1.1074e+00  1e-04  7e-16  2e-14
16: -1.1073e+00 -1.1073e+00  1e-06  6e-16  2e-14
17: -1.1073e+00 -1.1073e+00  1e-08  5e-16  2e-14
Optimal solution found.


In [ ]:
alpha = sol['x']
#print(alpha)
print(sol['primal objective'])

-1.1073109658574323


In [ ]:
complex_X = np.zeros((m,784))
for i in range(m):
  complex_X[i]= np_X_binary[i]*(np_Y_binary[i].item())
wt = np.dot(np.transpose(alpha),complex_X)
wt = np.transpose(wt)
#print(wt)
print(np.shape(wt))

(784, 1)


In [ ]:
# calculation for intercept
b=0.0
maxo=-1000000000.0
mino = 1000000000.0
for i in range(m):
  cont = np.dot(np.transpose(wt),np_X_binary[i])
  if np_Y_binary[i].item()==1:
    mino = min(mino,cont)
  else:
    maxo = max(maxo,cont)
b += maxo
b+= mino
b /= 2.0
print(b)

[0.49688294]


In [ ]:
# Test Set accuracy
df_inp_test = pd.read_csv("/content/drive/MyDrive/test.csv", header=None, usecols =arr)
df_labels_test = pd.read_csv("/content/drive/MyDrive/test.csv", header=None, usecols=arr_t)

In [ ]:
np_X_test = df_inp_test.to_numpy()
np_Y_test = df_labels_test.to_numpy()
print(np_X_test)
print(np_Y_test)
np_X_test /= 255

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.]
 [2.]
 [1.]
 ...
 [0.]
 [3.]
 [2.]]


In [ ]:
c= 0
for k in np_Y_test:
  if k==4 or k==5:
    c+=1
print(c)
np_X_bt = np.zeros((c,784))
np_Y_bt = np.zeros(c)
o=0
#print(np.shape(np_Y)[0])
for k in range(np.shape(np_Y_test)[0]):
  if np_Y_test[k]==4 or np_Y_test[k]==5:
    np_Y_bt[o]=np_Y_test[k]
    np_X_bt[o]=np_X_test[k]
    o+=1
print(np_X_bt)
print(np_Y_bt)
for i in range(np.shape(np_Y_bt)[0]):
  if np_Y_bt[i]==4:
    np_Y_bt[i]=1
  else:
    np_Y_bt[i]=-1
print(np_Y_bt)

1000
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[5. 4. 4. 5. 5. 5. 5. 5. 5. 5. 5. 4. 5. 4. 4. 5. 5. 4. 4. 5. 4. 4. 5. 4.
 5. 5. 5. 4. 5. 4. 4. 5. 4. 5. 5. 5. 4. 5. 5. 5. 4. 5. 5. 4. 5. 4. 4. 4.
 4. 4. 5. 5. 4. 5. 5. 5. 5. 5. 4. 4. 4. 5. 5. 5. 4. 4. 4. 5. 4. 5. 5. 5.
 5. 4. 4. 4. 4. 5. 5. 4. 4. 4. 4. 4. 4. 5. 4. 5. 5. 5. 4. 5. 5. 4. 5. 4.
 4. 5. 4. 5. 5. 5. 5. 5. 4. 4. 4. 5. 4. 5. 4. 4. 4. 5. 5. 4. 4. 5. 4. 4.
 5. 5. 5. 4. 5. 5. 4. 4. 5. 5. 4. 5. 5. 5. 5. 5. 4. 4. 4. 5. 5. 4. 5. 4.
 5. 5. 5. 5. 5. 5. 4. 4. 5. 5. 5. 5. 4. 4. 4. 5. 5. 5. 4. 4. 5. 4. 4. 5.
 4. 5. 5. 4. 5. 5. 5. 4. 4. 5. 4. 4. 5. 4. 5. 5. 5. 4. 4. 5. 4. 5. 4. 5.
 5. 5. 5. 5. 5. 4. 5. 5. 4. 5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 5. 4. 5. 4. 4.
 5. 4. 5. 4. 5. 4. 5. 5. 4. 4. 4. 4. 4. 4. 5. 5. 5. 5. 5. 5. 4. 5. 4. 5.
 4. 5. 5. 4. 4. 5. 4. 5. 5. 5. 4. 5. 4. 4. 4. 5. 4. 5. 4. 4. 5. 4. 4. 5.
 4. 4. 5. 4. 5. 5. 4. 5. 4. 5. 4. 4.

In [ ]:
#test accuracy
correct =0
for o in range(c):
  num = np.dot(np.transpose(wt),np_X_bt[o]).item()+b
  pred = -1.0
  if num>=0:
    pred =1.0
  if pred == np_Y_bt[o].item():
    correct+=1
print(correct/c)

0.984


Testing on validation set

In [ ]:
df_inp_val = pd.read_csv("/content/drive/MyDrive/val.csv", header=None, usecols =arr)
df_labels_val = pd.read_csv("/content/drive/MyDrive/val.csv", header=None, usecols=arr_t)

In [ ]:
np_X_val = df_inp_val.to_numpy()
np_Y_val = df_labels_val.to_numpy()
print(np_X_val)
print(np_Y_val)
np_X_val /= 255

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[2.]
 [0.]
 [5.]
 ...
 [4.]
 [0.]
 [2.]]


In [ ]:
c= 0
for k in np_Y_val:
  if k==4 or k==5:
    c+=1
print(c)
np_X_vt = np.zeros((c,784))
np_Y_vt = np.zeros(c)
o=0
#print(np.shape(np_Y)[0])
for k in range(np.shape(np_Y_val)[0]):
  if np_Y_val[k]==4 or np_Y_val[k]==5:
    np_Y_vt[o]=np_Y_val[k]
    np_X_vt[o]=np_X_val[k]
    o+=1
print(np_X_vt)
print(np_Y_vt)
for i in range(np.shape(np_Y_vt)[0]):
  if np_Y_vt[i]==4:
    np_Y_vt[i]=1
  else:
    np_Y_vt[i]=-1
print(np_Y_vt)

500
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[5. 4. 5. 5. 5. 5. 5. 4. 5. 4. 5. 4. 5. 4. 5. 4. 5. 5. 5. 4. 4. 5. 4. 4.
 4. 4. 4. 5. 4. 4. 4. 4. 4. 4. 4. 5. 5. 5. 4. 4. 5. 5. 4. 5. 5. 5. 5. 4.
 5. 4. 5. 5. 4. 5. 5. 5. 5. 5. 4. 4. 5. 5. 4. 4. 5. 4. 4. 5. 4. 5. 4. 4.
 5. 5. 4. 5. 4. 4. 5. 4. 5. 4. 4. 4. 5. 4. 4. 5. 5. 5. 5. 4. 4. 5. 5. 4.
 5. 5. 5. 4. 5. 4. 4. 4. 5. 4. 5. 5. 4. 5. 4. 4. 4. 5. 5. 4. 5. 4. 5. 4.
 4. 4. 5. 5. 5. 4. 4. 4. 4. 4. 4. 5. 5. 4. 5. 4. 4. 4. 5. 4. 5. 5. 4. 4.
 4. 5. 4. 4. 4. 4. 4. 4. 4. 5. 5. 5. 4. 5. 4. 4. 4. 4. 4. 4. 4. 4. 5. 4.
 5. 4. 5. 5. 5. 4. 4. 4. 5. 5. 5. 5. 5. 5. 4. 5. 5. 4. 5. 4. 4. 4. 5. 4.
 5. 4. 4. 5. 5. 5. 5. 5. 5. 4. 4. 4. 5. 5. 5. 5. 4. 4. 4. 4. 5. 4. 5. 5.
 5. 5. 4. 5. 5. 4. 5. 4. 5. 5. 4. 5. 5. 5. 5. 4. 5. 4. 4. 4. 4. 4. 5. 5.
 4. 4. 4. 4. 4. 5. 4. 5. 5. 5. 4. 5. 5. 4. 5. 5. 5. 5. 5. 4. 4. 5. 4. 5.
 4. 5. 4. 5. 5. 5. 5. 5. 5. 5. 5. 4. 

In [ ]:
#validation accuracy
correct =0
for o in range(c):
  num = np.dot(np.transpose(wt),np_X_vt[o]).item()+b
  pred = -1.0
  if num>=0:
    pred =1.0
  if pred == np_Y_vt[o].item():
    correct+=1
print(correct/c)

0.986


Gaussian Kernel

In [11]:
def kernel(np_arr0,np_arr1,gama):
  ut = math.exp(-1*gama*((np.linalg.norm((np_arr0-np_arr1),ord=None))**2))
  return ut

In [ ]:
print(kernel(np_X_binary[3],np_X_binary[2],gamma))

5.2236402548093205e-06


In [12]:
p_temp = np.zeros((m,m), dtype=np.float64)
gamma = 0.05
for i in range(m):
  for j in range(i+1):
    use = (np_Y_binary[i].item()*np_Y_binary[j].item())*(kernel(np_X_binary[i],np_X_binary[j],gamma))
    p_temp[i][j]=use
    p_temp[j][i]=use
print(p_temp)

[[ 1.00000000e+00 -1.00910740e-05 -3.52316787e-07 ...  6.26900053e-02
   2.40376717e-02 -3.04566654e-05]
 [-1.00910740e-05  1.00000000e+00  2.80870691e-02 ... -1.88034354e-05
  -1.31607721e-06  6.31432158e-02]
 [-3.52316787e-07  2.80870691e-02  1.00000000e+00 ... -9.08801115e-07
  -8.89604274e-08  6.08732200e-02]
 ...
 [ 6.26900053e-02 -1.88034354e-05 -9.08801115e-07 ...  1.00000000e+00
   1.10395714e-01 -8.81437019e-05]
 [ 2.40376717e-02 -1.31607721e-06 -8.89604274e-08 ...  1.10395714e-01
   1.00000000e+00 -1.13498587e-05]
 [-3.04566654e-05  6.31432158e-02  6.08732200e-02 ... -8.81437019e-05
  -1.13498587e-05  1.00000000e+00]]


In [ ]:
print(p_temp)

[[ 1.00000000e+00 -1.00910740e-05 -3.52316787e-07 ...  6.26900053e-02
   2.40376717e-02 -3.04566654e-05]
 [-1.00910740e-05  1.00000000e+00  2.80870691e-02 ... -1.88034354e-05
  -1.31607721e-06  6.31432158e-02]
 [-3.52316787e-07  2.80870691e-02  1.00000000e+00 ... -9.08801115e-07
  -8.89604274e-08  6.08732200e-02]
 ...
 [ 6.26900053e-02 -1.88034354e-05 -9.08801115e-07 ...  1.00000000e+00
   1.10395714e-01 -8.81437019e-05]
 [ 2.40376717e-02 -1.31607721e-06 -8.89604274e-08 ...  1.10395714e-01
   1.00000000e+00 -1.13498587e-05]
 [-3.04566654e-05  6.31432158e-02  6.08732200e-02 ... -8.81437019e-05
  -1.13498587e-05  1.00000000e+00]]


In [13]:
P = matrix(p_temp)
q = matrix(q_temp)
G = matrix(g_temp)
h = matrix(h_temp)
A = matrix(a_temp)
b = matrix(b_temp)

In [14]:
sol = solvers.qp(P,q,G,h,A,b)

     pcost       dcost       gap    pres   dres
 0: -1.5598e+02 -6.9927e+03  3e+04  2e+00  1e-15
 1: -8.8657e+01 -3.4857e+03  6e+03  2e-01  2e-15
 2: -5.9446e+01 -7.3609e+02  8e+02  2e-02  3e-15
 3: -1.0523e+02 -2.6151e+02  2e+02  4e-03  2e-15
 4: -1.2171e+02 -1.8069e+02  6e+01  1e-03  2e-15
 5: -1.2850e+02 -1.5673e+02  3e+01  2e-14  1e-15
 6: -1.3314e+02 -1.4204e+02  9e+00  8e-15  1e-15
 7: -1.3446e+02 -1.3849e+02  4e+00  1e-14  1e-15
 8: -1.3538e+02 -1.3651e+02  1e+00  3e-15  1e-15
 9: -1.3573e+02 -1.3581e+02  8e-02  1e-15  1e-15
10: -1.3576e+02 -1.3576e+02  2e-03  1e-14  1e-15
11: -1.3576e+02 -1.3576e+02  6e-05  1e-14  1e-15
Optimal solution found.


In [16]:
alpha_kernel = sol['x']
print(alpha_kernel)
#print(sol['primal objective'])

[ 5.95e-02]
[ 9.05e-08]
[ 1.09e-01]
[ 1.27e-01]
[ 3.98e-08]
[ 5.16e-01]
[ 3.19e-09]
[ 1.84e-08]
[ 8.64e-08]
[ 6.39e-01]
[ 1.89e-08]
[ 6.57e-09]
[ 4.28e-01]
[ 8.14e-03]
[ 1.79e-01]
[ 8.11e-09]
[ 1.66e-08]
[ 2.98e-08]
[ 5.33e-09]
[ 2.38e-01]
[ 1.09e-01]
[ 1.02e-01]
[ 2.23e-07]
[ 2.07e-08]
[ 8.46e-09]
[ 4.11e-08]
[ 3.76e-02]
[ 5.25e-08]
[ 4.47e-09]
[ 1.53e-08]
[ 1.62e-08]
[ 4.53e-09]
[ 1.09e-04]
[ 1.09e-08]
[ 7.96e-09]
[ 4.27e-09]
[ 4.76e-09]
[ 2.71e-01]
[ 1.91e-01]
[ 1.19e-07]
[ 7.19e-08]
[ 6.58e-08]
[ 9.79e-09]
[ 1.39e-07]
[ 4.04e-09]
[ 1.64e-08]
[ 7.85e-09]
[ 6.04e-01]
[ 1.56e-08]
[ 1.16e-08]
[ 1.33e-08]
[ 4.88e-09]
[ 1.79e-08]
[ 4.84e-09]
[ 9.29e-08]
[ 8.46e-09]
[ 5.29e-09]
[ 3.11e-08]
[ 1.79e-08]
[ 8.49e-09]
[ 3.23e-08]
[ 1.16e-02]
[ 4.74e-08]
[ 3.49e-04]
[ 6.96e-09]
[ 1.44e-08]
[ 1.10e-08]
[ 2.65e-08]
[ 3.16e-08]
[ 3.97e-09]
[ 2.51e-01]
[ 1.79e-08]
[ 3.72e-02]
[ 8.54e-02]
[ 1.11e-08]
[ 6.69e-01]
[ 1.15e-08]
[ 1.37e-08]
[ 2.24e-08]
[ 9.74e-05]
[ 3.21e-08]
[ 1.00e+00]
[ 3.14e-08]
[ 4.

In [ ]:
# sanity check
ko = np.dot(np.transpose(alpha_kernel),np_Y_binary)
print(ko)

[-1.30120163e-14]


In [17]:
def computation(alpha_kern,npYfunc,np_X_func,Xj,nums):
  s=0.0
  for i in range(nums):
    s += (alpha_kern[i])*(npYfunc[i].item())*kernel(np_X_func[i],Xj,gamma)
  return s

In [ ]:
print(computation(alpha_kernel,np_Y_binary,np_X_binary,np_X_binary[3],m))
print(np_Y_binary[3])

-0.6746363003512864
-1.0


In [ ]:
print(b_kernel)

[-0.06472907]


In [19]:
#getting new bias term
b_kernel = 0.0
maxo_k= -1000000.0
mino_k = 1000000.0
for i in range(m):
  cont = computation(alpha_kernel,np_Y_binary,np_X_binary,np_X_binary[i],m)
  if np_Y_binary[i].item()==1:
    mino_k = min(mino_k,cont)
  else:
    maxo_k = max(maxo_k,cont)
print(b_kernel)

0.0


In [20]:
b_kernel += maxo_k
b_kernel += mino_k
b_kernel /= 2.0
print(b_kernel)

-0.15048859573197398


In [21]:
print(m)

4500


In [23]:
#train acc.
correct_k =0
for o in range(m):
  num = 0.0 
  num += b_kernel
  #print(num)
  num += computation(alpha_kernel,np_Y_binary,np_X_binary,np_X_binary[o],m)
  #print(num, "yello")
  pred = -1.0
  if num>=0:
    pred =1.0
  if pred == np_Y_binary[o].item():
    correct_k+=1
print(correct_k/m)

1.0


In [25]:
print(correct_k/m)
print(b_kernel)

1.0
-0.15048859573197398


In [ ]:
#test accuracy
c= 0
for k in np_Y_test:
  if k==4 or k==5:
    c+=1
print(c)
correct_k =0
for o in range(c):
  num =0.0
  num += b_kernel.item()
  num += computation(alpha_kernel,np_Y_binary,np_X_binary,np_X_bt[o],m)
  pred = -1.0
  if num>=0:
    pred =1.0
  if pred == np_Y_bt[o].item():
    correct_k+=1
print(correct_k/c)

1000
1.0


In [ ]:
b_kernel=np.array([-0.06472907])

In [ ]:
#validation accuracy
c= 0
for k in np_Y_val:
  if k==4 or k==5:
    c+=1
print(c)
correct_k =0
for o in range(c):
  num = 0.0
  num += b_kernel.item()
  num += computation(alpha_kernel,np_Y_binary,np_X_binary,np_X_vt[o],m)
  pred = -1.0
  if num>=0:
    pred =1.0
  if pred == np_Y_vt[o].item():
    correct_k+=1
print(correct_k/c)

500
1.0
